In [1]:
%reset -f

In [2]:
import pandas as pd

In [68]:
outputDir = '../../nlp/data/nlp/output/'
outFile = 'nlpGOLDOutput.tsv'
inputDir = '../../nlp/data/input/'
inFile = 'biosamples-expanded.tsv'
#cols = ['BiosampleId', 'Biolink', 'BeginTerm', 'EndTerm', 'TokenizedTerm', 'PreferredTerm', 'CURIE', 'NaN1', 'S-Type', 'NaN2', 'UMLS_CUI']
ogerOutput = pd.read_csv(outputDir+outFile, sep='\t' )
ogerOutput = ogerOutput.drop(['BeginTerm', 'EndTerm'], axis = 1)
ogerOutput = ogerOutput.drop_duplicates()
ogerOutput.head()


,Study_BiosampleId,category,TokenizedTerm,PreferredTerm,CURIE
0,gold:Gb0101224,biolink:OntologyClass,Coal,coal,ENVO:02000091
1,gold:Gb0101224,biolink:OntologyClass,Coal Mine,coal mine,ENVO:00002169
4,gold:Gb0101224,biolink:ChemicalSubstance,water,water,CHEBI:15377
5,gold:Gb0101224,biolink:OntologyClass,water,liquid water,ENVO:00002006
6,gold:Gb0101225,biolink:OntologyClass,Coal,coal,ENVO:02000091


In [69]:
goldInput = pd.read_csv(inputDir+inFile, sep='\t', low_memory=False, usecols=['id', 'env_broad_scale', 'env_local_scale', 'env_medium'] )
goldInput.head()

,id,env_broad_scale,env_local_scale,env_medium
0,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792
1,gold:Gb0101225,ENVO:00002030,ENVO:00002169,ENVO:00005792
2,gold:Gb0101226,ENVO:00002030,ENVO:00002169,ENVO:00005792
3,gold:Gb0101227,ENVO:00002030,ENVO:00002169,ENVO:00005792
4,gold:Gb0101253,ENVO:00000446,ENVO:00002185,ENVO:00002985


In [77]:
inputOutputJoin = goldInput.merge(ogerOutput, left_on='id', right_on='Study_BiosampleId', how='right' )

In [78]:
inputOutputJoin = inputOutputJoin.drop(['Study_BiosampleId'], axis=1)
inputOutputJoin = inputOutputJoin.drop_duplicates()
inputOutputJoin.head()

,id,env_broad_scale,env_local_scale,env_medium,category,TokenizedTerm,PreferredTerm,CURIE
0,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal,coal,ENVO:02000091
1,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal Mine,coal mine,ENVO:00002169
2,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:ChemicalSubstance,water,water,CHEBI:15377
3,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,water,liquid water,ENVO:00002006
4,gold:Gb0101225,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal,coal,ENVO:02000091


In [79]:
inputOutputJoin.to_csv(outputDir+'inputOutputJoined.tsv', sep='\t', index=None)

In [80]:
filteredMatch = inputOutputJoin.query('CURIE == env_broad_scale' and \
                                      'CURIE == env_local_scale' and \
                                      'CURIE == env_medium')

filteredMatch = filteredMatch.drop_duplicates()

In [81]:
filteredMatch.head()

,id,env_broad_scale,env_local_scale,env_medium,category,TokenizedTerm,PreferredTerm,CURIE
18,gold:Gb0101253,ENVO:00000446,ENVO:00002185,ENVO:00002985,biolink:OntologyClass,oil,oil,ENVO:00002985
25,gold:Gb0101254,ENVO:00000446,ENVO:00002185,ENVO:00002985,biolink:OntologyClass,oil,oil,ENVO:00002985
1453,gold:Gb0102398,ENVO:00000447,ENVO:00000208,ENVO:03000033,biolink:OntologyClass,marine sediment,marine sediment,ENVO:03000033
1458,gold:Gb0102399,ENVO:00000447,ENVO:00000208,ENVO:03000033,biolink:OntologyClass,marine sediment,marine sediment,ENVO:03000033
1463,gold:Gb0102400,ENVO:00000447,ENVO:00000208,ENVO:03000033,biolink:OntologyClass,marine sediment,marine sediment,ENVO:03000033


In [82]:
filteredMatch.to_csv(outputDir+'filteredMatch.tsv', sep='\t', index=None)